In [1]:
import os

In [2]:
%pwd

'e:\\NutBodyslam053\\work\\Projects\\MLOps\\BD528-software_engineering\\research'

In [3]:
os.chdir("../")
%pwd

'e:\\NutBodyslam053\\work\\Projects\\MLOps\\BD528-software_engineering'

## entity

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth: int
    min_samples_leaf: int
    min_samples_split: int
    n_jobs: int
    target_column: str

## config

In [5]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH) -> None:

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            n_estimators=params.n_estimators,
            max_depth=params.max_depth,
            min_samples_leaf=params.min_samples_leaf,
            min_samples_split=params.min_samples_split,
            n_jobs=params.n_jobs,
            target_column=schema.name
        )

        return model_trainer_config

## components

In [6]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import joblib


class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig) -> None:
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop([self.config.target_column], axis=1)
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_train = train_data[self.config.target_column].ravel()
        y_test = test_data[self.config.target_column].ravel()

        rf = RandomForestClassifier(
            n_estimators=self.config.n_estimators,
            max_depth=self.config.max_depth,
            min_samples_leaf=self.config.min_samples_leaf,
            min_samples_split=self.config.min_samples_split,
            n_jobs=self.config.n_jobs,
            random_state=42
        )
        
        rf.fit(X_train, y_train)

        # Save trained model
        joblib.dump(rf, os.path.join(self.config.root_dir, self.config.model_name))

## pipeline

In [7]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

2023-10-31 14:56:34,105: INFO: common: yaml file: config\config.yaml loaded successfully
2023-10-31 14:56:34,107: INFO: common: yaml file: params.yaml loaded successfully
2023-10-31 14:56:34,110: INFO: common: yaml file: schema.yaml loaded successfully
2023-10-31 14:56:34,113: INFO: common: created directory at: artifacts
2023-10-31 14:56:34,114: INFO: common: created directory at: artifacts/04_model_trainer
